In [1]:
import os
git_dir = os.path.dirname(os.getcwd())
project_dir = os.path.join(git_dir,'zavmo')
os.chdir(project_dir)

In [2]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [7]:
from IPython.display import Markdown
import openai
from openai import OpenAI
from helpers.functions import create_pydantic_model, get_yaml_data, create_system_message
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [8]:
stage_config   = get_yaml_data('profile')
system_message_probe   = create_system_message('profile',stage_config,'probe')
system_message_extract = create_system_message('profile',stage_config,'extract')

In [195]:
system_content = """You are Zavmo, a highly interactive, supportive, and engaging AI assistant to help users learn and grow. Zavmo uses a 4-step learning process to help users learn and grow. The 4 Ds are Discover, Discuss, Deliver, and Demonstrate.

Before starting, you will need to know the user's profile details, this is the Profile Stage.

# Guidelines

- Friendly Engagement: Zavmo should adopt a warm, encouraging, and adaptable tone that reflects the user’s preferences and energy level, enhancing the enjoyment and reward of the learning experience.

- Structured Process: Guide users through a 4-step learning process: Discover, Discuss, Deliver, and Demonstrate. This structured path keeps users oriented, with Zavmo ensuring smooth transitions between these stages.

- Personalization: Tailor communication to individual user preferences, using their name and emojis to make the conversation more engaging and relatable.

- Motivation & Feedback: Reinforce user motivation with XP gains, fun phrases, emojis, and frequent praise. Track and celebrate user progress consistently to maintain engagement.

- Interaction Design: Engage users with well-paced interactions by asking one question at a time, encouraging them to think and respond.

- Emotional Sensitivity: Respond to emotional cues, adjusting pace and tone for increased support or to match user excitement. For neurodiverse users, slow down, provide double spacing, and offer extra encouragement.


The learner is currently in the **Profile** stage. 

Here is a list of instructions for each attribue that we need to probe the learner for in the current stage:

**first_name**: The first name of the user, often used to address them directly, e.g., 'Diana' or 'John'.
**last_name**:  The last name of the user, used to address them more formally or as part of their full name.
**age**:  The age of the user.
**edu_level**: The education level of the user, represented by a number  (1 for Primary School, 2 for Middle School, 3 for High School,  4 for Associate Degree, 5 for Bachelor's Degree, 6 for Master's Degree, 7 for PhD).
**current_role**: The current occupation or role of the user, indicating their professional or educational status,  e.g., 'Engineer', 'Teacher', 'Student', 'Doctor', 'Lawyer', 'Businessman', 'Artist', 'Musician',  'Writer', 'Scientist', 'Politician', 'Other'

You need to simultaneously talk to the learner, and also send notes to your assistant.
    - message: A personalized message to the user, probing for the attribute.
    - assistant:
        - note: A brief note instructing your assistant to take an action.
        - action: Must be one of `do_nothing` (if no new information has been presented), `update` (if new information is available), and `confirmed` (instruct the assistant that the conversation is complete so that the assistant can finalize)
        
"""

In [196]:
from enum import Enum
from typing import List
from pydantic import BaseModel, Field
from helpers.chat import force_tool_call

In [197]:
system_message = {'role':"system",'content':system_content}

In [177]:
user_content = "My name is Aditya Chhabra, I'm 28 years old, I have a Master's degree in Economics and I am a data scientist."

In [178]:
user_message = {"role":"user","content":user_content}

In [198]:
messages = [system_message,
            user_message
           ]

In [199]:
class action(str,Enum):
    do_nothing = 'do_nothing'
    update     = 'update'
    confirmed  = 'confirmed'
    
class assistant_note(BaseModel):
    note: str = Field(description='A short note for the assistant for context')
    action: action

class Response(BaseModel):
    message: str = Field(description='The personalized message to send to the learner.')
    assistant_note: assistant_note

In [206]:
from helpers.chat import summarize_stage_data, make_structured_call,summarize_history

In [201]:
from helpers.functions import *

In [189]:
messages.append({"role":"assistant","content":response.message})
messages.append({"role":"user","content":"Yes sounds good"})           

In [202]:
response = make_structured_call(Response, messages=messages,model='gpt-4o')

In [151]:
if response.action.value=='talk':
    result = force_tool_call(Talk, messages=messages,model='gpt-4o',tool_choice='required',parallel_tool_calls=False)    
    

In [124]:
profile_summary = summarize_stage_data(result.dict(),stage_name)

In [125]:
profile_summary

'**Profile**\n\n**first_name**: Aditya\n**last_name**: Chhabra\n**age**: 28\n**edu_level**: 6\n**current_role**: Data Scientist'

In [133]:
user_content = f"""Learner's input: {user_content}

**{response.action.value}**: {response.reason}

{profile_summary}
"""

In [135]:
response = force_tool_call(ActionReason, messages=messages,model='gpt-4o',tool_choice='required',parallel_tool_calls=True)

In [136]:
response.

ActionReason(reason='The user has provided all the required profile details in one statement, so I will summarize this information for review and confirmation.', action=<action.summarize: 'summarize'>)

In [41]:
response = openai_client.chat.completions.create(
                                      model='gpt-4o-mini',
                                        messages=messages,
                                      tools=[openai.pydantic_function_tool(tool) for tool in tools],
                                      tool_choice='required',
                                      parallel_tool_calls=False
                                     )

In [48]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_FYBAdf7hk7mQSwkp8Fmlo6vs', function=Function(arguments='{"attribute":"first_name","message":"Hi there! 😊 I\'d love to get to know you better. What\'s your first name?"}', name='probe'), type='function')]